In [1]:
#https://motius.breezy.hr/p/0b622aa1163e01-join-our-tech-community-as-working-student-m-f-d/apply?utm_source=pt-linkedin&utm_medium=job-posting

In [2]:
from transformers import  AutoModel,AutoTokenizer,AutoModelForMaskedLM , Trainer,TrainingArguments,\
BitsAndBytesConfig,pipeline,default_data_collator,DataCollatorWithPadding,DataCollatorForLanguageModeling
from transformers.utils import move_cache
from chat_template_utils import get_json_schema
from llama_cpp import Llama
from utils import *
from peft import *
import datasets
from datasets import Features, Value
import torchmetrics
import torch
from torch.utils.data.dataloader import DataLoader,Dataset
from hqq.engine.hf import HQQModelForCausalLM
from hqq.models.hf.base import AutoHQQHFModel
from huggingface_hub import snapshot_download
#import deepspeed
import os
import pandas as pd
import json
from var_dump import var_dump
from bs4 import BeautifulSoup,PageElement,Comment
import matplotlib.pyplot as plt
import seaborn as sns

cache_dir='/var/tmp/.cache/' #'/proj/ciptmp/ix05ogym/.cache/'
output_dir = cache_dir+'outputs/'

move_cache(cache_dir)


No ROCm runtime is found, using ROCM_HOME='/usr'
/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


0it [00:00, ?it/s]

In [3]:
train_dataset = pd.read_excel('train_forms.xlsx')[['input_text','labels']]
train_dataset.drop_duplicates(['input_text']).reset_index(drop=True)
c = train_dataset['labels']=='other'
train_dataset.loc[c,'labels']=pd.NA
c = train_dataset['labels']=='birthdate'
train_dataset.loc[c,'labels']='birth'

train_dataset['labels'].unique()

#c = train_dataset['labels']=='privacy'
#train_dataset.loc[c,:]

lll=[]
person = {"first name":"majid","last name":"sharghi foroushani","pronoun":"he","birth":"10/08/1997","age":"27","sex":"male","skill":"NLP,LLM","country":"Germany","address":"Philip-9","city":"Erlangen","phone number":"+497773154","available from":"6/24/2024","salary":"10000","German language level":"A1","English language level":"C1","visa":"do not need visa","email":"sharghi.majid@gmail.com","password":"123456","find us":"Google","xing":"www.xing.com","job":"I am stduent","university":"FAU","major":"AI","privacy":"privacy","search":"search","submit":"submit","cookie":"cookie","cover letter":"cover letter","linkedin":"www.linkedin.com","github":"www.github.com","login":"login","register":"register","cv":"/pah/to/cv.pdf","twitter":"www.twitter.com"}
for k,v in person.items():
    c =  train_dataset['labels']==k
    #train_dataset.loc[c,'labels']=k
    pass
    
print(train_dataset['labels'].value_counts())


labels
privacy                   63
search                    43
email                     38
cv                        37
submit                    28
first name                25
phone number              25
cover letter              24
linkedin                  23
last name                 21
find us                   15
cancel                    12
address                   11
salary                    10
available from            10
github                     9
question                   8
transcript                 8
recommendation             5
login                      5
birth                      5
password                   4
german language level      4
xing                       4
next                       4
website                    3
company                    3
house number               3
zip                        3
picture                    3
cookie                     3
city                       3
register                   3
eligible                   2
usernam

In [4]:
test_dataset = pd.read_excel('forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)
test_dataset['label'].unique()

array(['cv', 'first name', 'last name', 'email', 'phone number',
       'available from', 'github', 'privacy', 'submit', 'search',
       'country', 'sex', 'zip', 'cover letter', 'linkedin', 'visa',
       'xing', 'newsletter'], dtype=object)

In [5]:
test_dataset

,query,element,label,sudo_label
0,Personal information\n CV or resume\n Browse C...,"<input accept=""application/pdf,.pdf"" name=""cv""...",cv,login
1,Name *\n,"<input name=""first_name"" placeholder=""First"" r...",first name,first name
2,This field contains invalid characters\n,"<input name=""last_name"" placeholder=""Last"" req...",last name,transcript
3,This field contains invalid characters\n Email...,"<input name=""email"" placeholder=""yourmail@doma...",email,email
4,Phone\n,"<input name=""phone"" placeholder=""+49 176 123 4...",phone number,phone number
...,...,...,...,...
95,cover letter\n upload your cover letter\n,"<input accept=""application/msword, application...",cover letter,NaN
96,github\n,"<input name=""custom_attribute_356304"" placehol...",github,NaN
97,ihre e-mail-adresse\n,"<button aria-label=""subscribe for newsletter"" ...",newsletter,NaN
98,(!)\n this field is required\n,"<button id=""uploaddropboxresume"" type=""submit""...",submit,NaN


In [6]:
from sentence_transformers  import SentenceTransformer
#from transformers import AutoModel
model_name= 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
#sentence-transformers/all-MiniLM-L12-v2
embdeding_tokenizer = AutoTokenizer.from_pretrained(model_name,cache_dir=cache_dir)

#embeding_model= AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L12-v2',cache_dir=cache_dir)#.to('cuda')
embeding_model= SentenceTransformer(model_name,cache_folder=cache_dir)#.to('cuda')
#guide_model= SentenceTransformer(model_name,cache_folder=cache_dir)#.to('cuda')



embeding_model.eval()
embeding_model

/proj/ciptmp/ix05ogym/myenv/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)

In [7]:
data = pd.read_excel('query_element_4.xlsx')
data

,query,element,answer,label
0,keywords,"<input autocomplete=""off"" autofocus="""" id=""aut...",NaN,NaN
1,Email Address,"<input id=""fieldEmail"" name=""cm-ukutgu-ukutgu""...",sharghi.majid@gmail.com,email
2,Please tick to confirm that you consent to us ...,"<a href=""/privacy-policy"" target=""_blank"" titl...",privacy,privacy
3,which includes details on how you can withdraw...,"<input name=""gdpr_verification"" required="""" ty...",cookie,cookie
4,Subscribe,"<button type=""submit"">Subscribe</button>",subscribe,subscribe
...,...,...,...,...
1586,Please identify your race Please selectAmerica...,"<select aria-required=""false"" id=""job_applicat...",American Indian or Alaskan Native,ethnicity
1587,Race & Ethnicity Definitions,"<a href=""https://boards.cdn.greenhouse.io/docs...",A1,ethnicity
1588,Veteran Status Please selectI am not a protect...,"<select aria-required=""false"" id=""job_applicat...",I am not a protected veteran,NaN
1589,"Completing this form is voluntary, and we hope...","<a href=""https://www.dol.gov/ofccp"" target=""_b...",https://www.dol.gov/ofccp,NaN


In [8]:
#embeding_model = get_peft_model(embeding_model,LoraConfig(target_modules=['query','value']))
#embeding_model.print_trainable_parameters()


In [9]:

#c = train_dataset['labels']=='relocate'
#train_dataset.loc[c,'labels'] = 'willing to relocate'
#train_dataset[c]


In [10]:
#train_dataset.to_excel('train_forms.xlsx',index=0)

In [11]:
from sentence_transformers.trainer import SentenceTransformerTrainer,SentenceTransformerTrainingArguments
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


In [12]:
c = data['label'].notna()
data = data[c].reset_index(drop=True)
u = data['label'].unique()
id2key = dict(enumerate(u))
key2id = dict(zip(u,range(len(u))))
key2id

data.loc[:,'label'] = data['label'].map(lambda x:key2id[ x ])

In [13]:
#data.loc[:,'label']=1
features = Features({
    'query': Value('string'),
    'element': Value('string'),
    'answer': Value('string'),
    'label': Value('int64')
})
train_dataset = datasets.Dataset.from_pandas(data,features)

train_dataset = train_dataset.remove_columns(['answer'])#,'label'])


In [21]:
from sentence_transformers.losses import BatchSemiHardTripletLoss as myloss
trainer  =  SentenceTransformerTrainer(model=embeding_model,
                           train_dataset=train_dataset,
                           tokenizer=embdeding_tokenizer,
                           loss= myloss(embeding_model)#,384,len(u)),#,'sentence-transformers/all-MiniLM-L12-v2',True),
                           ,args=SentenceTransformerTrainingArguments(
                               output_dir=output_dir,
                               num_train_epochs=30,
                               per_device_train_batch_size=64,
                               logging_steps=10
                           )
                           
                           )


trainer.train()

Step,Training Loss
10,1.059200
20,1.000300
30,0.976200
40,0.981900
50,0.965900
60,0.824100
70,1.039100
80,0.930400
90,0.723400
100,0.919000


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

TrainOutput(global_step=510, training_loss=0.7586605305765189, metrics={'train_runtime': 66.8481, 'train_samples_per_second': 485.578, 'train_steps_per_second': 7.629, 'total_flos': 0.0, 'train_loss': 0.7586605305765189, 'epoch': 30.0})

In [15]:
#c = test_dataset['query'].isna()
#test_dataset.loc[c,'query']='cv'
#test_dataset.to_excel('forms.xlsx',index=0)

In [22]:
test_dataset = pd.read_excel('forms.xlsx')
c = test_dataset['label'].notna()
test_dataset = test_dataset.loc[c,:].reset_index(drop=True)

tp = test_dataset #train_dataset.to_pandas()
def compute_metric():
    embeding_model.eval()
    acc=0
    
    with torch.no_grad():
        
        l=embeding_model.encode(LABEL_INDEX_TO_KEY)
        l = torch.tensor(l)
        print(l.shape)
        
        for i,x in tp.iterrows():
            #print(x)
            #for i,x in train_dataset.iterrows():
            #te = embdeding_tokenizer([x['element']],max_length=512,truncation=True,return_tensors='pt')['input_ids']
            #print(i)
            #x['label']
            o = embeding_model.encode( [x['query']] )
            o2 = embeding_model.encode( x['element'] )
            
            o = torch.tensor(o)
            o2 = torch.tensor(o2)
            
            sim = torch.cosine_similarity(l,o)
            #print(o2.shape)
            #sim = torch.cosine_similarity(o2,o,0)

            m = sim.argmax()
            tp.loc[i,'sudo_label']=LABEL_INDEX_TO_KEY[m]
            #tp.loc[i,'sudo_label_value']=sim
            if LABEL_INDEX_TO_KEY[m]==x['label']:
                acc+=1
                pass
            #print(LABEL_INDEX_TO_KEY[sim.argmax()],x['label'])
            
           
             
    
    acc/=len(test_dataset)
    print("acc:",acc)
    
    
    
compute_metric()

torch.Size([61, 384])
acc: 0.43


In [17]:
test_dataset.loc[11,'query']='cv *'
tp

,query,element,label,sudo_label
0,Personal information\n CV or resume\n Browse C...,"<input accept=""application/pdf,.pdf"" name=""cv""...",cv,github
1,Name *\n,"<input name=""first_name"" placeholder=""First"" r...",first name,first name
2,This field contains invalid characters\n,"<input name=""last_name"" placeholder=""Last"" req...",last name,pronoun
3,This field contains invalid characters\n Email...,"<input name=""email"" placeholder=""yourmail@doma...",email,email
4,Phone\n,"<input name=""phone"" placeholder=""+49 176 123 4...",phone number,phone number
...,...,...,...,...
95,cover letter\n upload your cover letter\n,"<input accept=""application/msword, application...",cover letter,cover letter
96,github\n,"<input name=""custom_attribute_356304"" placehol...",github,github
97,ihre e-mail-adresse\n,"<button aria-label=""subscribe for newsletter"" ...",newsletter,email
98,(!)\n this field is required\n,"<button id=""uploaddropboxresume"" type=""submit""...",submit,find us


In [18]:
for i,x in tp.iterrows():
            print(x['query'])
            print(x['element'])
            print(x['sudo_label'])
            #print(x['sudo_label_value'])
            print('-----------------------------------')

Personal information
 CV or resume
 Browse CV to autocomplete your application. You can still fill your profile manually.
 
<input accept="application/pdf,.pdf" name="cv" tabindex="-1" type="file"/>
github
-----------------------------------
Name *
 
<input name="first_name" placeholder="First" required="" type="text"/>
first name
-----------------------------------
This field contains invalid characters
 
<input name="last_name" placeholder="Last" required="" type="text"/>
pronoun
-----------------------------------
This field contains invalid characters
 Email *
 
<input name="email" placeholder="yourmail@domain.com" required="" type="email"/>
email
-----------------------------------
Phone
 
<input name="phone" placeholder="+49 176 123 4455" type="text"/>
phone number
-----------------------------------
Available from *
 
<input name="available_from" placeholder="" required="" type="text"/>
available from
-----------------------------------
GitHub
 
<input name="custom_attribute_529